In [8]:
import os
import pandas as pd
import numpy as np
from alpha_vantage.timeseries import TimeSeries
import csv

In [9]:
key = 'E4M6BVAOLPUG2OL7'
ts = TimeSeries(key, output_format='pandas')

In [12]:
data, googl_meta_data = ts.get_daily('GOOGL', outputsize='full')
data

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2019-11-27,1315.42,1317.64,1309.4742,1310.620,282622.0
2019-11-26,1309.91,1314.01,1304.0400,1313.000,940545.0
2019-11-25,1296.26,1310.41,1295.6100,1305.640,1179658.0
2019-11-22,1303.00,1306.90,1289.3600,1293.670,1388912.0
2019-11-21,1299.25,1311.12,1290.7500,1300.140,1277338.0
...,...,...,...,...,...
2004-08-25,104.76,108.00,103.8800,106.000,9188600.0
2004-08-24,111.24,111.60,103.5700,104.870,15247300.0
2004-08-23,110.76,113.48,109.0500,109.400,18256100.0


In [13]:
path = str(os.getcwd()) + "/" + "data"
data.to_csv("trainData.csv")

In [22]:
def read_train():
    prices = pd.read_csv("trainData.csv")
    prices = clean(prices)
    return prices

def clean(prices):
    (m, n) = prices.shape

    prices = prices.loc[:, ['date', 'open', 'close']]

    # Add new column "Close Price at the Previous Day" and "Open Price at the Previous Day"
    prevColumn = prices.loc[:, ['open', 'close']].assign(index = lambda x: x.index-1)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev')

    # Add new column "Close Price at the Day Before" and "Open Price at the Day Before"
    prevColumn = prices.loc[:, ['open', 'close']].assign(index = lambda x: x.index-2)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev2')

    prevColumn = prices.loc[:, ['open', 'close']].assign(index = lambda x: x.index-3)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev3')

    prevColumn = prices.loc[:, ['open', 'close']].assign(index = lambda x: x.index-4)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev4')

    prevColumn = prices.loc[:, ['open', 'close']].assign(index = lambda x: x.index-5)
    prevColumn = prevColumn.set_index('index')
    prices = prices.join(prevColumn, rsuffix='Prev5')
    
    prices = prices[:m-5]
    return prices


def write_train(data):
    data.loc[:, ['date','open','OpenPrev','ClosePrev','OpenPrev2','ClosePrev2','OpenPrev3','ClosePrev3',
                 'OpenPrev4','ClosePrev4','OpenPrev5','ClosePrev5']].to_csv("trainX.csv", index=False)
    data.loc[:, ['close']].to_csv( "trainY.csv",index=False)
    
def write_test(data):
    print(year_Y_2017)
    year_2017=data.loc[mask]
    year_2017.loc[:, ['date','open','OpenPrev','ClosePrev','OpenPrev2','ClosePrev2','OpenPrev3','ClosePrev3',
                 'OpenPrev4','ClosePrev4','OpenPrev5','ClosePrev5']].to_csv("trainX.csv", index=False)
    year_2017.loc[:, ['close']].to_csv( "trainY.csv",index=False)

In [23]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score

In [24]:
def read_val():
    X = pd.read_csv("trainX.csv", header=0)
    y = pd.read_csv("trainY.csv", header=0)
    X['date'] = pd.to_datetime(X['date'], format='%Y-%m-%d')
    return X, y



def plot_trend(x, predict, y, title):
    plt.xticks(rotation=45)
    plt.plot_date(x, y, fmt='b-', xdate=True, ydate=False, label='Real value')
    plt.plot_date(x, predict, fmt='r-', xdate=True, ydate=False, label='Predicted value')
    plt.legend(loc='upper center')
    plt.ylabel('Close prices')
    plt.title(title)
    plt.grid()
    plt.show()


def modify(X, columns):
    columns.append('Date2num')
    returnX = X.loc[:, columns]
    return returnX


def predict_and_plot(X, date):
    m, n = X.shape
    X_test = X.loc[np.floor(m*0.6)+1:]
    X_train = X.loc[:np.floor(m*0.6)]
    
    y_test = y.loc[np.floor(m*0.6)+1:]
    y_train = y.loc[:np.floor(m*0.6)]
    
    date_test = date.loc[np.floor(m*0.6)+1:]
    date_train = date.loc[:np.floor(m*0.6)]
    
    lr = LinearRegression()

    lr.fit(X_train, y_train)

    print('Coefficients: \n', lr.coef_)
    print('Intercept: \n', lr.intercept_)
    pred = lr.predict(X_test)
    return lr,pred


In [25]:
prices_train=read_train()
write_train(prices_train)

X, y = read_val()

X['Date2num'] = X['date'].apply(lambda x: mdates.date2num(x))
date = X.loc[:, ['date']]
del X['date']

X1 = modify(X, [])
predict_and_plot(X1,date)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [36]:
#X2 = modify(X, ['Open'])
#lr,close_val=predict_and_plot(X2, date)

X3 = modify(X, ['open','OpenPrev','ClosePrev'])
lr,close_val=predict_and_plot(X3, date)
X3

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [34]:
daterange = pd.date_range('2019-01-01', datetime.today().strftime('%Y-%m-%d'), freq='B')
newDate = pd.DataFrame(data=daterange, columns=['date'])
newDate = newDate.dropna()
newDate

,date
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-07
...,...
232,2019-11-21
233,2019-11-22
234,2019-11-25
235,2019-11-26


In [35]:
prevOpen = data.loc[:,['open']]
prevOpen = prevOpen.iloc[len(prevOpen)-1]
prevClose = close_val.item(len(close_val)-1)
prevOpen = prevOpen.values.item(0)
newX3 = X3
try:
    newX3.drop('Date2num',axis=1,inplace=True)
except:
    print('already dropped')
dummy = pd.DataFrame([[prevClose,prevOpen,prevClose]], columns=['open','OpenPrev','ClosePrev'])
newX3 = newX3.append(dummy, ignore_index=True)
date = date.append(newDate, ignore_index=True)
for x in range(0,len(newDate)-1):
    print(len(newX3))
    print(len(date))
    lr,close_val = predict_and_plot(newX3, date)
    prevClose = close_val.item(len(close_val)-1)
    prevOpen = newX3.loc[:,['open']]
    prevOpen = prevOpen.iloc[len(prevOpen)-1]
    prevOpen = prevOpen.values.item(0)
    dummy = pd.DataFrame([[prevClose,prevOpen,prevClose]], columns=['open','OpenPrev','ClosePrev'])
    newX3 = newX3.append(dummy, ignore_index=True)

newX3

KeyError: "None of [Index(['open'], dtype='object')] are in the [columns]"

In [27]:
date


,Date
0,1980-12-12
1,1980-12-15
2,1980-12-16
3,1980-12-17
4,1980-12-18
...,...
9436,2018-04-24
9437,2018-04-25
9438,2018-04-26
9439,2018-04-27
